# IMPORT LIBS

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
import time
import mplfinance as mpf
import os
import datetime

# CONSTANTS


In [15]:
GRAPH_EXTEND = 10
RISK_REWARD_RATIO = 1
POSITION_TP_SL_RANGE = 20
CISD_CHECK_RANGE = 11

# Preprocess

In [16]:
# Dataset path
path1 = r"C:\Users\Nasip-PC\Desktop\Dosyalar\Kodlar\Das Reich\AlgoV2\Datasets\NASDAQ\USATECH_ADJUSTED_30.09.2021-21.09.2024.csv"
path2 = r"C:\Users\Nasip-PC\Desktop\Dosyalar\Kodlar\Das Reich\AlgoV2\Datasets\NASDAQ\USATECH_ADJUSTED_29.09.2018-29.09.2021.csv"
path3 = r"C:\Users\Nasip-PC\Desktop\Dosyalar\Kodlar\Das Reich\AlgoV2\Datasets\NASDAQ\USATECH.IDXUSD_Candlestick_1_M_BID_23.09.2024-30.11.2024.csv"
# Read the dataset

df = pd.read_csv(path1)

# # 'Local time' kolonundaki zaman dilimini temizle ve datetime formatına dönüştür
# df['Local time'] = df['Local time'].str.split(' GMT').str[0]  # ' GMT+0300' kısmını çıkarır
# df['Local time'] = pd.to_datetime(df['Local time'], format='%d.%m.%Y %H:%M:%S.%f')  # Doğru formatı belirtin
df['Local time'] = pd.to_datetime(df['Local time'], format='%Y-%m-%d %H:%M:%S')

# Set the index to 'Local time' for mplfinance compatibility
df_chart = df.copy()
df_chart.set_index('Local time', inplace=True)


# DRAW CHART

In [17]:
def calculate_max_drawdown(values):
    peak = values[0]
    max_drawdown = 0
    
    for value in values:
        # Update the peak value if the current value is greater than the previous peak
        peak = max(peak, value)
        
        # Calculate the drawdown at the current value
        drawdown = (peak - value) 
        
        # Update the max drawdown if the current drawdown is larger
        max_drawdown = max(max_drawdown, drawdown)
    
    return max_drawdown


In [18]:
# LIQ SEVIYELERİNİ ÇİZEN KOD
def draw_liqs(df, start_time, end_time, low_liqs = [], high_liqs = [], current_idx=None, title="", save_path = None):
    """Creates a candlestick chart with swing liquidity prices and current price."""
    
    df_filtered = df[(df['Local time'] >= start_time) & (df['Local time'] <= end_time)].copy()
    df_filtered['Local time'] = pd.to_datetime(df_filtered['Local time'])
    df_filtered.set_index('Local time', inplace=True)
    
    add_plots = []
    if low_liqs != [] :
        for low in low_liqs:
            add_plots.append(mpf.make_addplot(pd.Series(low, index=df_filtered.index), color='red', linestyle=':'))
    
    if high_liqs != [] :
        for high in high_liqs:
            add_plots.append(mpf.make_addplot(pd.Series(high, index=df_filtered.index), color='green', linestyle=':'))

        # Save each plot as a PNG frame
    if save_path != None:
        mpf.plot(df_filtered, type='candle', volume=False,  style='charles', addplot=add_plots,savefig=save_path)

    if save_path == None:
        mpf.plot(df_filtered, type='candle', volume=False, style='charles', addplot=add_plots)



# POZISYON BİTİNCE SONUCU ÇİZEN KOD
def draw_position(df_chart, start_local_time='2021-09-30 01:00:00', end_local_time='2021-10-03 01:00:00', 
                  current_value=None, high_value=None, low_value=None, trade_hit="STOP", save_path = None):
    """Draws a candlestick chart with entry, take profit, and stop loss prices."""
    start_time = pd.to_datetime(start_local_time)
    end_time = pd.to_datetime(end_local_time)

    df_filtered = df_chart[(df_chart.index >= start_time) & (df_chart.index <= end_time)]
    
    addplot_list = []

    if current_value != None:
        addplot_list.append(mpf.make_addplot([current_value] * len(df_filtered), color='black', linestyle='--', label='Entry Price'))


    if high_value != None:
        addplot_list.append(mpf.make_addplot([high_value] * len(df_filtered), color='green', linestyle='--', label='Take Profit'))


    if low_value != None:
        addplot_list.append(mpf.make_addplot([low_value] * len(df_filtered), color='red', linestyle='--', label='Stop Loss'))

    if save_path != None:
        mpf.plot(df_filtered, type='candle', volume=False, title=trade_hit, style='charles', addplot=addplot_list,savefig=save_path)

    if save_path == None:
        mpf.plot(df_filtered, type='candle', volume=False, title=trade_hit, style='charles', addplot=addplot_list)


def print_iteration(df, current_index, decimal = 10000):
    if current_index % decimal == 0:
        print("Index Left : " ,len(df) - current_index)


# CONTROL FUNCTIONS

In [19]:
def check_for_fvg(df, current_index):
    
    """ 
    BULLISH FVG -> 1 
    BEARISH FVG -> - 1 
    NO FVG -> 0 
    
    
    """
    
    # Get the previous candle's high and low
    prev_2_high = df.iloc[current_index - 2]['High']
    prev_2_low = df.iloc[current_index - 2]['Low']
    
    # Get the current candle's high and low
    current_high = df.iloc[current_index]['High']
    current_low = df.iloc[current_index]['Low']

    fvg_body_threshold = 1
    fvg_wick_threshold = 1
    
    current_body = abs(df.iloc[current_index - 1]['Close'] - df.iloc[current_index - 1]['Open'])
    
    # Check for a Bullish FVG (current candle's low is higher than the previous candle's high)
    if current_body > fvg_body_threshold and current_low - prev_2_high > fvg_wick_threshold:
        return 1
    
    # Check for a Bearish FVG (current candle's high is lower than the previous candle's low)
    if current_body > fvg_body_threshold and prev_2_low - current_high > fvg_wick_threshold:
        return - 1
    
    return 0 

In [20]:
def is_bullish(df, current_index):
    return df.iloc[current_index]["Close"] - df.iloc[current_index]["Open"] > 0

In [21]:
def if_long_cisd(df,current_index = None, cisd_check_range = CISD_CHECK_RANGE):
    """
    BU FONKSİYON ÖN KOŞUL OLARAK SON LIQ SEVİYESİNİN KIRILDIĞINI VARSAYAR
    """
    
    """ LIQ ALINMADAN ÖNCEKİ SON YEŞİL MUMU BULUR """

    bearish_candle_start_idx = current_index
    while is_bullish(df, bearish_candle_start_idx):
        bearish_candle_start_idx -= 1

    bullish_candle_start_idx = bearish_candle_start_idx
    while not is_bullish(df, bullish_candle_start_idx):
        bullish_candle_start_idx -= 1

    cisd_level = df.iloc[bullish_candle_start_idx]["Close"]

    #print("FİRST CISD : ", df.iloc[bullish_candle_start_idx]["Local time"])

    # OB HAS FOUND. CONTINUE WITH CONFIRMATION
    is_confirmed = False
    for confirmation_index in range(current_index + 1, current_index + cisd_check_range):
        if df.iloc[confirmation_index]["Close"] > cisd_level:
            ### OB LEVEL CONFIRMED. OPEN LONG HERE
            is_confirmed = True
            #print("LAST CISD : ", df.iloc[bullish_candle_start_idx]["Local time"])

            break   
            
        bearish_candle_start_idx = confirmation_index
        while is_bullish(df, bearish_candle_start_idx):
            bearish_candle_start_idx -= 1

        bullish_candle_start_idx = bearish_candle_start_idx
        while not is_bullish(df, bullish_candle_start_idx):
            bullish_candle_start_idx -= 1

        cisd_level = df.iloc[bullish_candle_start_idx]["Close"]

    return is_confirmed, cisd_level, confirmation_index 

In [22]:
def if_short_cisd(df,current_index = None, cisd_check_range = CISD_CHECK_RANGE):
    """
    BU FONKSİYON ÖN KOŞUL OLARAK SON LIQ SEVİYESİNİN KIRILDIĞINI VARSAYAR
    """
    
    """ LIQ ALINMADAN ÖNCEKİ SON KIRMIZI MUMU BULUR """

    bullish_candle_start_idx = current_index
    while not is_bullish(df, bullish_candle_start_idx):
        bullish_candle_start_idx -= 1

    bearish_candle_start_idx = bullish_candle_start_idx
    while is_bullish(df, bearish_candle_start_idx):
        bearish_candle_start_idx -= 1

    cisd_level = df.iloc[bearish_candle_start_idx]["Close"]

    #print("FİRST CISD : ", df.iloc[bullish_candle_start_idx]["Local time"])

    # OB HAS FOUND. CONTINUE WITH CONFIRMATION
    is_confirmed = False
    for confirmation_index in range(current_index + 1, current_index + cisd_check_range):
        if df.iloc[confirmation_index]["Close"] < cisd_level:
            ### OB LEVEL CONFIRMED. OPEN LONG HERE
            is_confirmed = True
            #print("LAST CISD : ", df.iloc[bullish_candle_start_idx]["Local time"])

            break   
            
            
        bullish_candle_start_idx = current_index
        while not is_bullish(df, bullish_candle_start_idx):
            bullish_candle_start_idx -= 1

        bearish_candle_start_idx = bullish_candle_start_idx
        while is_bullish(df, bearish_candle_start_idx):
            bearish_candle_start_idx -= 1

        cisd_level = df.iloc[bearish_candle_start_idx]["Close"]


    return is_confirmed, cisd_level, confirmation_index 

# POSITION

In [23]:
# LONG İŞLEM SONRASI DURUMU KONTROL EDEN KOD
def open_long_position(df, df_chart, entry_index,fvg_index, entry_price, balance, tp_counter, sl_counter, lot_size,total_profit_loss, 
                            visualize = True, position_tp_sl_range = POSITION_TP_SL_RANGE, risk_reward_ration = RISK_REWARD_RATIO,graph_extend = GRAPH_EXTEND, total_rr = None):
    
    stop_price = df.iloc[entry_index - position_tp_sl_range : entry_index + 1]["Low"].min()
    tp_price = entry_price + abs(entry_price - stop_price) * risk_reward_ration


    """  ADJUST tp_price, stop_price LEVELS IN HERE """


    for trade_index in range(entry_index + 1, len(df)):
        if df.iloc[trade_index]["Low"] < stop_price:
            change_rate = 1 - (abs(entry_price - stop_price) / entry_price)
            balance *= change_rate
            sl_counter += 1
            
            total_rr -= 1
        
            total_profit_loss = total_profit_loss - (abs(stop_price - entry_price) * lot_size ) 

            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = stop_price, high_value = tp_price, current_value = entry_price, trade_hit = "LONG SL")
            break

        elif df.iloc[trade_index]["High"] > tp_price:

            change_rate = 1 + (abs(tp_price - entry_price) / entry_price)
            balance *= change_rate
            tp_counter += 1

            total_rr += risk_reward_ration

            total_profit_loss = total_profit_loss + (abs(tp_price - entry_price) * lot_size ) 

            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = stop_price, high_value = tp_price, current_value = entry_price, trade_hit = "LONG TP")

            break
        
        # BURDA FVG'Yİ KIRDIĞI İÇİN İŞLEMİ KAPATIYORUZ
        elif df.iloc[trade_index]["Close"] < df.iloc[fvg_index - 2]["High"]:
            change_rate = 1 - (abs(entry_price - df.iloc[trade_index]["Close"]) / entry_price)
            balance *= change_rate
            sl_counter += 1

            total_rr -= abs(entry_price - df.iloc[trade_index]["Close"]) / (abs(entry_price - stop_price))

            total_profit_loss = total_profit_loss - (abs(stop_price - entry_price) * lot_size ) 


            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = df.iloc[trade_index]["Close"], high_value = tp_price, current_value = entry_price, trade_hit = "LONG SL")
            #print("BREAK THE CYCLE MORTY")
            
            break

    return balance, tp_counter , sl_counter, trade_index, stop_price, tp_price,change_rate, total_profit_loss, total_rr

In [24]:
# SHORT İŞLEM SONRASI DURUMU KONTROL EDEN KOD
def open_short_position(df, df_chart, entry_index,fvg_index, entry_price, balance, tp_counter, sl_counter, lot_size,total_profit_loss, 
                            visualize = True, position_tp_sl_range = POSITION_TP_SL_RANGE, risk_reward_ration = RISK_REWARD_RATIO,graph_extend = GRAPH_EXTEND, total_rr = None):
    
    stop_price = df.iloc[entry_index - position_tp_sl_range : entry_index + 1]["High"].max()
    tp_price = entry_price - abs(stop_price - entry_price) * risk_reward_ration


    for trade_index in range(entry_index + 1, len(df)):
        if df.iloc[trade_index]["High"] > stop_price:
            change_rate = 1 - (abs(stop_price - entry_price) / entry_price)
            balance *= change_rate
            sl_counter += 1
            total_rr -= 1
            total_profit_loss = total_profit_loss - (abs(stop_price - entry_price) * lot_size ) 

            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = stop_price, high_value = tp_price, current_value = entry_price, trade_hit = "SHORT SL")
            break

        elif df.iloc[trade_index]["Low"] < tp_price:

            change_rate = 1 + (abs(entry_price - tp_price) / entry_price)
            balance *= change_rate
            tp_counter += 1

            total_rr += risk_reward_ration

            total_profit_loss = total_profit_loss + (abs(tp_price - entry_price) * lot_size ) 

            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = stop_price , high_value = tp_price, current_value = entry_price, trade_hit = "SHORT TP")

            break
        

        # BURDA FVG'Yİ KIRDIĞI İÇİN İŞLEMİ KAPATIYORUZ
        elif df.iloc[trade_index]["Close"] > df.iloc[fvg_index - 2]["Low"]:
            change_rate = 1 - (abs(df.iloc[trade_index]["Close"] - entry_price) / entry_price)
            balance *= change_rate
            sl_counter += 1

            total_profit_loss = total_profit_loss - (abs(stop_price - entry_price) * lot_size )
            total_rr -= abs(entry_price - df.iloc[trade_index]["Close"]) / (abs(entry_price - stop_price))

            if visualize:
                draw_position(df_chart,df.iloc[fvg_index - graph_extend]["Local time"], df.iloc[trade_index + graph_extend]["Local time"],low_value = df.iloc[trade_index]["Close"], high_value = tp_price, current_value = entry_price, trade_hit = "SHORT SL")
            #print("BREAK THE CYCLE MORTY")
            
            break
        


    return balance, tp_counter , sl_counter, trade_index, stop_price, tp_price,change_rate, total_profit_loss, total_rr

# Backtest

In [29]:
current_index = 100
balance = 100
lot_size = 0.01
total_profit_loss = 0

tp_counter = 0
sl_counter = 0
balances = [balance]
pl_values = []

tp_trades = []
sl_trades = []

low_liqs = []
high_liqs = []

long_trades = 0
short_trades = 0
RR = 1

fr_range = 10
total_rr = 0
total_rr_list = [total_rr]
# Define the start and end time for the check
start_time = datetime.time(00, 00)  # 16:30
end_time = datetime.time(23, 59)  # 23:00

df_for_model = df.copy()
df_for_model["Position Type"] = 0
df_for_model["Is Profitable"] = 0

visualize = False

while current_index < len(df) - 1000:
    current_index += 1
    # BU DATA ÜZERİNDE KAÇ İNDEKSİN KALDIĞINI GÖSTERİYOR
    # print_iteration(df, current_index)

    # Get the current time at the current index
    current_time = df['Local time'].iloc[current_index].time()  # Get time part of the datetime

    # Check if the current time is between start_time and end_time
    if start_time <= current_time <= end_time:

        if check_for_fvg(df, current_index) == 1:
            """ LONG POSITION """
            
            fvg_index = current_index - 1 
            fvg_upper_bound = df.iloc[current_index]["Low"]
            fvg_lower_bound = df.iloc[current_index - 2]["High"]

            fr_check_range = current_index + fr_range

            while current_index < fr_check_range:
                current_index += 1

                if df.iloc[current_index]["Close"] < fvg_lower_bound:
                    break

                if check_for_fvg(df,current_index) == 1:
                    current_index -= 1
                    break


                if df.iloc[current_index]["Low"] <  (fvg_upper_bound + fvg_lower_bound) / 2:
                        long_trades += 1
                        # LİMİT EMRİ BULLİSH FVG HİGH BOUND'A ATIYORUZ
                        entry_price = df.iloc[current_index]["Close"]
                        entry_index = current_index

                        tp_prev = tp_counter

                        balance, tp_counter , sl_counter, trade_index, stop_price, tp_price,change_rate, total_profit_loss, total_rr   = open_long_position(df, df_chart, entry_index,fvg_index, entry_price, balance, 
                                tp_counter, sl_counter, lot_size,total_profit_loss, 
                                visualize = visualize, position_tp_sl_range = POSITION_TP_SL_RANGE, risk_reward_ration = RISK_REWARD_RATIO,graph_extend = GRAPH_EXTEND, total_rr = total_rr )


                        title = F"""
LONG
FVG Time : {df.iloc[fvg_index]['Local time']} , Entry Time : {df.iloc[current_index]['Local time']} , Exit Time : {df.iloc[trade_index]['Local time']} 
Entry Price : {entry_price}, Stop Price : {stop_price}, Take Profit Price : {tp_price}
TOTAL RR : {total_rr} Drawdown : {calculate_max_drawdown(total_rr_list):.2f}
    """
                        print(title)

                        balances.append(balance)
                        total_rr_list.append(total_rr)
                        current_index = trade_index

                        df_for_model.loc[entry_index, "Position Type"] = 1
                        
                        if tp_prev == tp_counter:    
                            df_for_model.loc[entry_index, "Is Profitable"] = -1
                        else:
                            df_for_model.loc[entry_index, "Is Profitable"] = 1


                        break

        elif check_for_fvg(df, current_index) == -1:
            """ SHORT POSITION  """

            fvg_index = current_index - 1

            fvg_lower_bound = df.iloc[current_index]["High"]
            fvg_upper_bound = df.iloc[current_index - 2]["Low"]


            fr_check_range = current_index + fr_range

            while current_index < fr_check_range:
                current_index += 1

                if df.iloc[current_index]["Close"] > fvg_upper_bound:
                    break

                if check_for_fvg(df,current_index) == -1:
                    current_index -= 1
                    break

                if df.iloc[current_index]["High"] > (fvg_upper_bound + fvg_lower_bound) / 2:
                    
                        short_trades += 1
                        # LİMİT EMRİ BULLİSH FVG HİGH BOUND'A ATIYORUZ
                        entry_price = df.iloc[current_index]["Close"]
                        entry_index = current_index

                        tp_prev = tp_counter

                        balance, tp_counter , sl_counter, trade_index, stop_price, tp_price,change_rate, total_profit_loss, total_rr   = open_short_position(df, df_chart, entry_index,fvg_index, entry_price, balance, 
                                tp_counter, sl_counter, lot_size,total_profit_loss, 
                                visualize = visualize, position_tp_sl_range = POSITION_TP_SL_RANGE, risk_reward_ration = RISK_REWARD_RATIO,graph_extend = GRAPH_EXTEND, total_rr = total_rr )


                        title = F"""
SHORT
FVG Time : {df.iloc[fvg_index]['Local time']} , Entry Time : {df.iloc[current_index]['Local time']} , Exit Time : {df.iloc[trade_index]['Local time']} 
Entry Price : {entry_price}, Stop Price : {stop_price}, Take Profit Price : {tp_price}
TOTAL RR : {total_rr} Drawdown : {calculate_max_drawdown(total_rr_list):.2f}
    """
                        print(title)

                        balances.append(balance)
                        total_rr_list.append(total_rr)
                        current_index = trade_index


                        df_for_model.loc[entry_index, "Position Type"] = -1
                        
                        if tp_prev == tp_counter:    
                            df_for_model.loc[entry_index, "Is Profitable"] = -1
                        else:
                            df_for_model.loc[entry_index, "Is Profitable"] = 1

                        break

title = F"TOTAL RR: {total_rr:.2f} TP:{tp_counter} SL:{sl_counter} WIN RATE:{(tp_counter / (tp_counter + sl_counter)):.2f}Drawdown: {calculate_max_drawdown(total_rr_list):.2f}"
plt.figure(figsize=(10, 5))  # Set the figure siz;e
plt.plot(total_rr_list, linestyle='-', color='black', label='Balances')
plt.xlabel("Index", fontsize=12)
plt.ylabel("Value", fontsize=12)
plt.title(title)
plt.grid(True)  # Add a grid for better readability

# Save the plot as an image
plt.savefig(fr"C:\Users\Nasip-PC\Desktop\Dosyalar\Kodlar\Das Reich\AlgoV2\Algorithms\FVG_CISD\FVG\1m_Long_SHORT_CISD.png", dpi=300)  # Save with high resolution
plt.show()  # Display the plot

df.to_csv(r"C:\Users\Nasip-PC\Desktop\Dosyalar\Kodlar\Das Reich\AlgoV2\Algorithms\Data intensive\fvg_mean_threshold_model.csv")



SHORT
FVG Time : 2021-09-30 01:58:00 , Entry Time : 2021-09-30 02:02:00 , Exit Time : 2021-09-30 02:03:00 
Entry Price : 14800.349, Stop Price : 14806.599, Take Profit Price : 14794.099
TOTAL RR : -1 Drawdown : 0.00
    

SHORT
FVG Time : 2021-09-30 02:06:00 , Entry Time : 2021-09-30 02:08:00 , Exit Time : 2021-09-30 02:09:00 
Entry Price : 14803.889, Stop Price : 14808.039, Take Profit Price : 14799.738999999998
TOTAL RR : -1.149397590361148 Drawdown : 1.00
    

LONG
FVG Time : 2021-09-30 02:10:00 , Entry Time : 2021-09-30 02:12:00 , Exit Time : 2021-09-30 02:15:00 
Entry Price : 14806.209, Stop Price : 14794.019, Take Profit Price : 14818.399000000001
TOTAL RR : -1.757272897990315 Drawdown : 1.15
    

SHORT
FVG Time : 2021-09-30 02:15:00 , Entry Time : 2021-09-30 02:17:00 , Exit Time : 2021-09-30 02:18:00 
Entry Price : 14803.739, Stop Price : 14808.059, Take Profit Price : 14799.419
TOTAL RR : -2.757272897990315 Drawdown : 1.76
    

LONG
FVG Time : 2021-09-30 02:19:00 , Entry Ti

KeyboardInterrupt: 